# Etape 2 : feature engineering

In [ ]:
### 2.1. Choix des nouveaux features :

Afin d'**avoir un modèle performant**, nous allons **ajouter d'autres features**. En effet, la localisation, le prix, le nombre de pièces et la surface ne donnent pas assez d'informations pour bien valoriser un bien immobilier.

Selon BSI Economics, **trois facteurs** qui influencent les prix de l'immobilier :

- **l'environnement économique et financier** (offre et demande de logement, contexte économie, structure des marchés immobiliers, logements neufs et anciens, démographie, taux de logements vacants) ;

- **les conditions d'emprunts** (taux variables/fixes, maturités des prêts, ratio emprunt sur valeur, répartition des crédits, solvabilité, système de garantie) ;

- **l'environnement fiscal** (mesures fiscales incitatives à la location, distorsion fiscale des locataires vers les propriétaires).

En particulier, **un agent immobilier** observe, par exemple, les features suivant :
* prix au m2 du quartier (le projet qui nous a inspiré utilise l'algorithme BallTree pour l'obtenir), 
* PIB du département (mesure de l'activité économique des alentours), 
* densité du département (mesure de l'urbanisation, de l'activité), 
* variation de population non naturelle (mesure de l'attractivité),
* mois de la transaction (saisonnalité des prix dans certaines régions touristiques), 
* taux de logements vacants dans la région (mesure de l'attractivité).

### 2.2. Importation des nouveaux features :

In [ ]:
# Creation de la variable prix_m2 :
data['prix_m2'] = data['surface_terrain'] / data['valeur_fonciere']

Ces features ont été **générées dans le notebook "external_data"**. Nous invitons le lecteur à s'y référer afin d'apprécier la manière dont ils ont été importés puis préprocessés.

In [ ]:
# Importation des données externes INSEE :

external_data = retrieve_data()
external_data.head(3)

In [ ]:
list_numbers = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
data["code_departement"] = data["code_departement"].apply(
    lambda dep: str(dep) if str(dep) not in list_numbers else "0"+str(dep))
data.head(3)

In [ ]:
dep_vente = data["code_departement"].unique()
dep_commun = external_data["code_dep"].unique()

print(f"Département présents dans les ventes mais pas dans nos données complémentaires : \
      {[code_dep for code_dep in dep_vente if code_dep not in dep_commun]}")
print(f"Département présents dans les données complémentaires mais pas dans les ventes : \
{[code_dep for code_dep in dep_commun if code_dep not in dep_vente]}")
print(f"Nombre de départements uniques dans notre table générale : {len(data['code_departement'].unique())}.")

In [ ]:
display("Nombre de lignes correspondant aux Landes qui vont être perdues")
display(data[data["code_departement"] == '40']["code_departement"].count())
display("Longueur avant inner join :")
len_pre_join = data.shape[0]
display(data.shape[0])

### 2.3. Jointure des deux tables de données :

In [ ]:
data = data.set_index("code_departement").join(external_data.set_index("code_dep"), how = "inner", rsuffix = "jo")
data = data.reset_index().rename(columns = {"index": "code_departement"})
display("Longueur après inner join :")
display(data.shape[0])
len_post_join = data.shape[0]
display("Lignes perdues pendant la jointure (du fait de l'inner join) :")
display(abs(len_post_join - len_pre_join))

L'inner join avec les autres features nous fait perdre les 1799 lignes correspondant aux ventes dans le département 40, comme on pouvait s'y attendre.
La base de donnée est bien cleanée avec **les nouveaux features ajoutés**.